In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from imripy import merger_system as ms
from imripy import halo, inspiral, waveform, plot_utils

In [ ]:
m1 = 1e3 * ms.solar_mass_to_pc
m2 = 1. *  ms.solar_mass_to_pc

In [ ]:
M_d = 1.
R_d = 1.
z_d = 1.

mnDisc = halo.MiyamotoNagaiDisc(M_d, R_d, z_d)

In [ ]:
rho6 = 5.448e15 * ms.solar_mass_to_pc  # in 1/pc^2 
alpha = 7./3.
spike = halo.Spike.FromRho6(rho6, m1, alpha)

In [ ]:
r_grid = np.geomspace(1e-10, 1e5)
l, = plt.plot(r_grid, mnDisc.density(r_grid), label='z=0')
plt.plot(r_grid, mnDisc.mass(r_grid), color = l.get_c(), linestyle='--')
l, = plt.plot(r_grid, mnDisc.density(r_grid, z=1.), label='z=1')
plt.plot(r_grid, mnDisc.mass(r_grid, z=1.), color = l.get_c(), linestyle='--')
l, = plt.plot(r_grid, mnDisc.density(r_grid, z=2.), label='z=2')
plt.plot(r_grid, mnDisc.mass(r_grid, z=2), color = l.get_c(), linestyle='--')

plt.plot(r_grid, spike.density(r_grid), label='spike')
plt.xscale('log'); plt.yscale('log'); plt.grid(); plt.legend()

In [ ]:
sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.))
sp_b = ms.SystemProp(m1, m2, halo.ConstHalo(0.), baryonicHalo = mnDisc)
sp_dm = ms.SystemProp(m1, m2, spike)
sp_dm_b = ms.SystemProp(m1, m2, spike, baryonicHalo = mnDisc)

In [ ]:
a0 = 100. * sp_0.r_isco()
e0 = 0.1

opt_0 = inspiral.Classic.EvolutionOptions(acc=1e-10, verbose=1)
baryonicOptions = inspiral.Classic.EvolutionOptions(acc=1e-10, verbose=3, accretion=True, accretionModel='Bondi-Hoyle')
opt_b = inspiral.Classic.EvolutionOptions(acc=1e-10, verbose=3, baryonicHaloEffects=True, baryonicEvolutionOptions=baryonicOptions)
opt_dm = inspiral.Classic.EvolutionOptions(acc=1e-10, verbose=1)
opt_dm_b = opt_b

In [ ]:
ev_0 = inspiral.Classic.Evolve(sp_0, a0, e0, opt=opt_0)
ev_b = inspiral.Classic.Evolve(sp_b, a0, e0, opt=opt_b)

In [ ]:
ev_dm = inspiral.Classic.Evolve(sp_dm, a0, e0, opt=opt_dm)
ev_dm_b = inspiral.Classic.Evolve(sp_dm_b, a0, e0, opt=opt_dm_b)

In [ ]:
fig_ae, axes_ae = plt.subplots(2, 1, figsize=(6,10))
fig_gw, axes_gw = plt.subplots(2, 1, sharex='col', figsize=(6,10))

ax_m = axes_ae[0].twinx()
l = plot_utils.plotEvolution(sp_0, ev_0, ax_a=axes_ae[0], ax_ae=axes_ae[1], label='vacuum')
plot_utils.plotGWcharacteristicStrain(sp_0, ev_0, axes_gw[0], label='vacuum' , harmonics=[2,3], color=l.get_c())

l = plot_utils.plotEvolution(sp_b, ev_b, ax_a=axes_ae[0], ax_ae=axes_ae[1], ax_m=ax_m, m2=m2, label='baryonic')
plot_utils.plotGWcharacteristicStrain(sp_b, ev_b, axes_gw[0], label='baryonic' , harmonics=[2,3], color=l.get_c())
plot_utils.plotDeltaN(sp_0, ev_0, sp_b, ev_b, axes_gw[1], color=l.get_c(), label='baryonic')

l = plot_utils.plotEvolution(sp_dm, ev_dm, ax_a=axes_ae[0], ax_ae=axes_ae[1], label='spike')
plot_utils.plotGWcharacteristicStrain(sp_dm, ev_dm, axes_gw[0], label='dm' , harmonics=[2,3], color=l.get_c())
plot_utils.plotDeltaN(sp_0, ev_0, sp_dm, ev_dm, axes_gw[1], color=l.get_c(), label='spike')

l = plot_utils.plotEvolution(sp_dm_b, ev_dm_b, ax_a=axes_ae[0], ax_ae=axes_ae[1], ax_m=ax_m, m2=m2, label='spike+baryonic')
plot_utils.plotGWcharacteristicStrain(sp_dm_b, ev_dm_b, axes_gw[0], label='spike+baryonic' , harmonics=[2,3], color=l.get_c())
plot_utils.plotDeltaN(sp_0, ev_0, sp_dm_b, ev_dm_b, axes_gw[1], color=l.get_c(), label='spike+baryonic')

plot_utils.plotDeltaN(sp_dm, ev_dm, sp_dm_b, ev_dm_b, axes_gw[1], color=l.get_c(), label='spike vs baryonic', linestyle='--')

for ax in np.append(axes_ae.flatten(), axes_gw.flatten()):
    ax.grid()
    ax.legend()